In [2]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms, ToTensor
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torchvision.models as models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop((224,224)),
    ToTensor()
])

train_dataset = CIFAR10(
    root="../data",
    train = True,
    transform=transform,
)

test_dataset = CIFAR10(
    root = "../data",
    train =  True,
    transform= transform
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle=False, num_workers=0)



/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <EB3FF92A-5EB1-3EE8-AF8B-5923C1265422> /Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/jtappen/.local/share/mamba/envs/ml/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can igno

In [5]:
model = models.vgg11_bn(weights=models.VGG11_BN_Weights.DEFAULT)
for param in model.parameters():
    param.requires_grad = False

model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 2)

loss_fn = nn.HuberLoss()
optim = torch.optim.Adam(model.classifier[-1].parameters(), lr=0.001)

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [11]:
def train(model, optim, loss_fn, dataloader):
    model.train()
    epoch_loss, samples = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        targets = y.float().unsqueeze(1)  * (0.1)
        targets = torch.cat([targets, targets], dim = 1)
    
        optim.zero_grad()
        preds = model(X)
        loss = loss_fn(preds, targets)

        loss.backward()
        optim.step()

        num_samples = y.size(0)
        epoch_loss += loss.item()
        samples += num_samples

        if batch % 100 == 0:
            print(f"Batch {batch} -------------------")
            print(f"Batch Loss = {epoch_loss/(batch+1)}")

    return (epoch_loss/samples)

for epoch in (range(5)):
    avg_loss = train(model, optim, loss_fn, train_loader)
    print(f"EPOCH LOSS: {avg_loss}") 







KeyboardInterrupt: 